In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE42977"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE42977"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE42977.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE42977.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE42977.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sequential Binary Gene-Ratio Tests Define a Novel Molecular Diagnostic Strategy for Malignant Pleural Mesothelioma"
!Series_summary	"The gene-expression ratio  technique was used to design a molecular signature to diagnose MPM from among other potentially confounding diagnoses and differentiate the epithelioid from the sarcomatoid histological subtype of MPM."
!Series_overall_design	"Microarray analysis was performed on 113 specimens including MPMs and a spectrum of tumors and benign tissues comprising the differential diagnosis of MPM.  A sequential combination of binary gene-expression ratio tests was developed to discriminate MPM from other thoracic malignancies .  This method was compared to other bioinformatic tools and this signature was validated  in an independent set of 170 samples.  Functional enrichment analysis was performed to identify differentially expressed probes."
Sample Characteristics Dictionary:
{0: ['tissue: control', 'tissue

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Dict, Any, Optional, Callable, List

# 1. Assess gene expression data availability
# This dataset appears to have gene expression data as it mentions microarray analysis
# and gene-expression ratio tests in the background information
is_gene_available = True

# 2.1. Identify data availability and rows
# Examining the sample characteristics dictionary
sample_chars = {0: ['tissue: control', 'tissue: Spindle Cell Sarcoma', 'tissue: Sarcoma', 'tissue: Metastatic Melanoma', 'tissue: Pleomorphic Sarcoma', 'tissue: Renal Cell Carcinoma-Clear Cell', 'tissue: Synovial Sarcoma', 'tissue: Metastatic Thymoma', 'tissue: Metastatic Prostate Cancer', 'tissue: Stomach Cancer-Stromal Sarcoma', 'tissue: Non-Hodgkins Lymphoma', 'tissue: Hemangioendothelioma', 'tissue: Papillary Thyroid Carcinoma', 'tissue: Metastatic Thyroid Cancer', 'tissue: Lymphocytic Lymphoma', 'tissue: Thymoma', 'tissue: Melanoma-Malignant', 'tissue: Hemangiopericytoma', 'tissue: Thyroid Carcinoma', 'tissue: Monophasic Synovial Sarcoma', 'tissue: Metastatic Alveolar Soft Part Sarcoma', 'tissue: Metastatic Meningeal Hemangiopericytoma', 'tissue: Follicular Lymphoma', 'tissue: Rhabdomyosarcoma', 'tissue: Myofibrosarcoma', 'tissue: Renal Cell Carcinoma - Chromophobe', 'tissue: MPM Epithelioid', 'tissue: MPM Sarcomatoid', 'tissue: MPM Biphasic', 'tissue: Normal Lung']}

# For Kidney_Chromophobe trait, we look for "Renal Cell Carcinoma - Chromophobe" in the tissue type
trait_row = 0  # The row index containing tissue type information
age_row = None  # No age information available
gender_row = None  # No gender information available

# 2.2. Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert tissue type to binary: 1 for Kidney Chromophobe, 0 for others
    """
    if value is None or ':' not in value:
        return None
        
    value_part = value.split(':', 1)[1].strip()
    
    # Check if the value indicates Kidney Chromophobe (RCC-Chromophobe)
    if 'Renal Cell Carcinoma - Chromophobe' in value_part:
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """
    This is a placeholder function since age data is not available
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    This is a placeholder function since gender data is not available
    """
    return None

# 3. Save metadata about the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Assuming clinical_data is a DataFrame with sample characteristics
    # We need to first create this DataFrame from the sample_chars dictionary
    clinical_data = pd.DataFrame()
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            clinical_data.loc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [0.0], 1: [0.0], 2: [0.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [0.0], 7: [0.0], 8: [0.0], 9: [0.0], 10: [0.0], 11: [0.0], 12: [0.0], 13: [0.0], 14: [0.0], 15: [0.0], 16: [0.0], 17: [0.0], 18: [0.0], 19: [0.0], 20: [0.0], 21: [0.0], 22: [0.0], 23: [0.0], 24: [0.0], 25: [1.0], 26: [0.0], 27: [0.0], 28: [0.0], 29: [0.0]}
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE42977.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 46713 rows
First 20 gene IDs:
Index(['ILMN_10000', 'ILMN_100000', 'ILMN_100007', 'ILMN_100009', 'ILMN_10001',
       'ILMN_100010', 'ILMN_10002', 'ILMN_100028', 'ILMN_100030',
       'ILMN_100031', 'ILMN_100034', 'ILMN_100037', 'ILMN_10004', 'ILMN_10005',
       'ILMN_100054', 'ILMN_100059', 'ILMN_10006', 'ILMN_100075',
       'ILMN_100079', 'ILMN_100083'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene IDs shown in the output, these are Illumina probe IDs (starting with "ILMN_") 
# and not standard human gene symbols. These probe IDs need to be mapped to human gene symbols 
# for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5512256 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'GB_ACC': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'SEQUENCE': ['CTCTCTAAAGGGACAACAGAGTGGACAGTCAAGGAACTCCACATATTCAT', 'GGGGTCAAGCCCAGGTGAAATGTGGATTGGAAAAGTGCTTCCCTTGCCCC', 'CCAGGCTGTAAAAGCAAAACCTCGTATCAGCTCTGGAACAATACCTGCAG', 'CCAGACAGGAAGCATCAAGCCCTTCAGGAAAGAATATGCGAGAGTGCTGC', 'TGTGCAGAAAGCTGATGGAAGGGAGAAAGAATGGAAGTGGGTCACACAGC'], 'Definition': ['UI-CF-EC0-abi-c-12-0-UI.s1 UI-CF-EC0 Homo sapiens cDNA clone UI-CF-EC0-abi-c-12-0-UI 3, mRNA sequence', 'PREDICTED: Homo sapiens similar to spectrin domain with coiled-coils 1 (LOC441782), mRNA.', 'Homo sapiens jumonji domain containing 1A (JMJD1A), mRNA.', 'hi56g05.x1 Soares_NFL_T_GBC_S1 Homo sapiens cDNA clone IMAGE:2976344 3, mRNA sequence', 'wk77d04.x1 NCI_CGAP_Pan1 Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# The gene expression data uses Illumina probe IDs (e.g., "ILMN_10000")
# From the gene annotation preview, we can see that:
# - The 'ID' column contains the same Illumina probe IDs
# - The 'Symbol' column contains the gene symbols we need for mapping

# 2. Extract the mapping columns from the gene annotation dataframe
print("Creating gene mapping between probe IDs and gene symbols...")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Extracted mapping data with {len(mapping_df)} rows")

# Preview the mapping data
print("\nMapping data preview:")
print(mapping_df.head())

# Check how many unique probe IDs and genes we have
print(f"\nNumber of unique probe IDs in mapping: {mapping_df['ID'].nunique()}")
print(f"Number of unique genes in mapping: {mapping_df['Gene'].nunique()}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Generated gene expression data with {len(gene_data)} genes and {gene_data.shape[1]} samples")

# Preview the gene expression data
print("\nGene expression data preview (first few genes):")
print(gene_data.head())


Creating gene mapping between probe IDs and gene symbols...


Extracted mapping data with 28566 rows

Mapping data preview:
           ID       Gene
1  ILMN_35826  LOC441782
2  ILMN_25544     JMJD1A
6  ILMN_10519      NCOA3
8  ILMN_17234  LOC389834
9  ILMN_19244   C17orf77

Number of unique probe IDs in mapping: 28566
Number of unique genes in mapping: 24998

Converting probe-level measurements to gene expression data...
Generated gene expression data with 18401 genes and 117 samples

Gene expression data preview (first few genes):
         GSM1054230  GSM1054231  GSM1054232  GSM1054233  GSM1054234  \
Gene                                                                  
A1BG       0.903276    0.821580    0.865428    0.890772    0.986434   
A2BP1      1.719921    8.432093    1.797852    6.514504    1.802560   
A2M       27.154642   17.070914   34.490890    9.773921   31.282731   
A2ML1      0.893226    0.963602    0.996714    1.108153    0.855068   
A3GALT2    1.357179    1.423179    1.518340    1.352150    1.415984   

         GSM1054235  GSM10

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Load the previously saved clinical data
print("\nLoading previously saved clinical data...")
try:
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        print("Clinical data preview:")
        print(preview_df(clinical_df))
        is_trait_available = True
    else:
        print("Clinical data file not found, using data from previous step")
        clinical_df = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error loading clinical features: {e}")
    is_trait_available = False
    clinical_df = None

# 3. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if clinical_df is not None and not normalized_gene_data.empty:
        # Analyze samples in both datasets
        clinical_samples = clinical_df.columns.tolist()
        genetic_samples = normalized_gene_data.columns.tolist()
        print(f"Clinical data has {len(clinical_samples)} samples")
        print(f"Genetic data has {len(genetic_samples)} samples")
        
        # Find common samples
        common_samples = [s for s in clinical_samples if s in genetic_samples]
        print(f"Found {len(common_samples)} common samples between datasets")
        
        if len(common_samples) == 0:
            # Sample ID formats might be different, but could match positions
            # Compare a few IDs to check format differences
            if len(clinical_samples) > 0 and len(genetic_samples) > 0:
                print(f"Clinical sample format example: {clinical_samples[0]}")
                print(f"Genetic sample format example: {genetic_samples[0]}")
                print("No matching sample IDs found - formats appear to be different")
            
            # Use sample positions for linking if possible
            if len(clinical_samples) <= len(genetic_samples):
                # Rename genetic samples to match clinical format
                genetic_subset = normalized_gene_data.iloc[:, :len(clinical_samples)]
                genetic_subset.columns = clinical_samples
                
                # Now link with renamed samples
                linked_data = geo_link_clinical_genetic_data(clinical_df, genetic_subset)
                print(f"Linked data by sample position: shape {linked_data.shape}")
            else:
                print("Cannot link data: clinical samples exceed genetic samples")
                linked_data = pd.DataFrame()
        else:
            # Use common samples for linking
            clinical_subset = clinical_df[common_samples]
            genetic_subset = normalized_gene_data[common_samples]
            linked_data = geo_link_clinical_genetic_data(clinical_subset, genetic_subset)
            print(f"Linked data with common samples: shape {linked_data.shape}")
        
        # Check for trait column
        if linked_data.shape[0] > 0:
            if trait in linked_data.columns:
                trait_count = linked_data[trait].count()
                print(f"Number of samples with trait values: {trait_count}")
                
                if trait_count > 0:
                    # Customize missing value handling for this dataset
                    print("\nCustomizing missing value handling...")
                    
                    # Simple approach: drop samples with missing trait values
                    linked_data = linked_data.dropna(subset=[trait])
                    
                    # Ensure trait is correctly formatted as numeric
                    linked_data[trait] = pd.to_numeric(linked_data[trait], errors='coerce')
                    
                    # For columns with majority missing values, drop them
                    missing_pct = linked_data.isna().mean()
                    keep_cols = [col for col in linked_data.columns if missing_pct[col] < 0.5]
                    linked_data = linked_data[keep_cols]
                    
                    # Fill remaining missing values
                    linked_data = linked_data.fillna(linked_data.mean())
                    
                    print(f"After customized missing value handling, data shape: {linked_data.shape}")
                    
                    if linked_data.shape[0] > 0:
                        # Check for bias in features
                        print("\nChecking for bias in features...")
                        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                    else:
                        print("Error: All samples were removed during missing value handling.")
                        is_biased = True
                else:
                    print("No samples have valid trait values. Dataset cannot be used.")
                    is_biased = True
            else:
                print(f"Error: Trait column '{trait}' not found in linked data")
                print(f"Available columns: {linked_data.columns.tolist()[:10]}...")
                is_biased = True
        else:
            print("Linked data is empty. Dataset cannot be used.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    import traceback
    traceback.print_exc()
    linked_data = pd.DataFrame()
    is_biased = True

# 4. Final quality validation
print("\nPerforming final validation...")
try:
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased if 'is_biased' in locals() else True,
        df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
        note="Dataset contains normalized gene expression data but has limitations in clinical annotations and sample matching"
    )
except Exception as e:
    print(f"Error in final validation: {e}")
    is_usable = False

# 5. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...


After normalization: 17207 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE42977.csv

Loading previously saved clinical data...
Loaded clinical data with shape: (1, 30)
Clinical data preview:
{'0': [0.0], '1': [0.0], '2': [0.0], '3': [0.0], '4': [0.0], '5': [0.0], '6': [0.0], '7': [0.0], '8': [0.0], '9': [0.0], '10': [0.0], '11': [0.0], '12': [0.0], '13': [0.0], '14': [0.0], '15': [0.0], '16': [0.0], '17': [0.0], '18': [0.0], '19': [0.0], '20': [0.0], '21': [0.0], '22': [0.0], '23': [0.0], '24': [0.0], '25': [1.0], '26': [0.0], '27': [0.0], '28': [0.0], '29': [0.0]}

Linking clinical and genetic data...
Clinical data has 30 samples
Genetic data has 117 samples
Found 0 common samples between datasets
Clinical sample format example: 0
Genetic sample format example: GSM1054230
No matching sample IDs found - formats appear to be different
Linked data by sample position: shape (30, 17208)
Number of samples with trait values: 30

Customizing missing value handling...


After customized missing value handling, data shape: (30, 17208)

Checking for bias in features...
For the feature 'Kidney_Chromophobe', the least common label is '1.0' with 1 occurrences. This represents 3.33% of the dataset.
The distribution of the feature 'Kidney_Chromophobe' in this dataset is severely biased.


Performing final validation...
Dataset not usable for Kidney_Chromophobe association studies. Data not saved.
